# Projeto 1 - Ciência dos Dados

Nome: Lucca Martins Pereira Lima

Nome: Christian Delfs

Nome: Vitor de Moraes Caixeta

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
c:\Users\Vitor Caixeta\OneDrive\C_dados DP\Nova pasta\22-2b-cd-p1-grupo_luccampl\notebooks


Carregando a base de dados com as notícias classificadas manualmente:

In [3]:
filename = 'dados.xlsx'
dados = pd.read_excel(filename)

In [4]:
train = pd.read_excel(filename, sheet_name = 'Treinamento')
train.head(5)

,Categoria,Titulo,Descrição,Data,Pagina,Target
0,tecnologia,Pesquisadores encontram fóssil do maior inseto...,Paleontologistas ingleses descobriram o fóssil...,25/12/2021 16:51,66.0,2.0
1,tecnologia,WhatsApp define “head” no Brasil de olho em el...,O WhatsApp anunciou em março que o Brasil seri...,04/04/2022 17:39,37.0,1.0
2,tecnologia,Os tropeços da vacina Sputnik V na América Latina,A Sputnik V foi a primeira vacina contra a cov...,04/08/2021 17:03,121.0,0.0
3,tecnologia,Variante britânica do coronavírus aponta uma m...,"A variante britânica do coronavírus, considera...",22/01/2021 14:08,238.0,0.0
4,tecnologia,Vacinação contra a covid-19 na Venezuela: espe...,Yaritza de Velázquez se preparava para visitar...,05/05/2021 17:07,169.0,0.0


In [5]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Categoria,Titulo,Descrição,Data,Pagina,Target
0,tecnologia,Eclipse lunar ‘quase total’ na próxima noite s...,A América do Norte e grande parte da América d...,18/11/2021 23:00,80,2
1,tecnologia,"Pandemia vai acabar em 1 ano, diz CEO da farma...","O CEO da Moderna, Stéphane Bancel, declarou qu...",24/09/2021 12:06,99,0
2,tecnologia,Moderna acredita que sua dose de reforço funci...,A farmacêutica americana Moderna disse nesta s...,21/12/2021 03:35,68,0
3,tecnologia,Mais alto e por mais tempo: helicóptero Ingenu...,"Ingenuity, o pequeno helicóptero da NASA, cons...",22/04/2021 15:01,177,1
4,tecnologia,Argentina pede que vacinas contra a covid-19 s...,"O presidente da Argentina, Alberto Fernández, ...",28/01/2021 19:25,233,0


___
## Classificador automático


Faça aqui uma descrição do seu assunto e o que considerou como relevante ou não relevante na classificação das notícias (Target).

ESCREVA AQUI...

Nossa proposta no projeto, se

Para o nosso projeto, com o tema de 'Tecnologia', retirado do site 'Isto é Dinheiro', a target escolhida terá como foco associar e classificar qual o tipo da notícia, usando como fatores de classificação: "notícias relacionadas à saúde", "notícias relacionadas ao avanço de tecnologias" e "outras". 

0 - Notícias relacionadas à saúde

1 - Notícias relacionadas ao avanço de tecnologias

2 - Notícias que não se relacionam à nenhum dos 2 (outras)

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [6]:
import re 


def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[!-.:?;123456789%]'
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

In [7]:
filtro_0 = train.Target == 0
dados_target_0 = train.loc[filtro_0, :].reset_index(drop=True)

filtro_1 = train.Target == 1
dados_target_1 = train.loc[filtro_1, :].reset_index(drop=True)

filtro_2 = train.Target == 2
dados_target_2 = train.loc[filtro_2, :].reset_index(drop=True)

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações das notícias entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nas notícias. Ex: stemming, lemmatization, stopwords
* CONSIDEROU mais de duas caterogias na variável Target e INCREMENTOU a quantidade de notícias, mantendo pelo menos 250 notícias por caterogia (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* Para Target com duas categorias: CRIOU pelo menos quatro categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto (pelo menos dois cenários, exceto aqueles já apresentados em sala pelos professores: por exemplo, filtro de spam)
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item Qualidade do Classificador a partir de novas separações das Notícias entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**